In [22]:
# selenium
from selenium import webdriver

In [63]:
url = "https://boardgamearena.com/gamepanel?game=azul"
driver = webdriver.Firefox()
driver.maximize_window()

driver.get(url)
current_window = driver.current_window_handle

In [64]:
# click on div: class="text-center bgabutton bgabutton_blue bgabutton_no_margin bgabutton_huge svelte-g0rhr9"
from selenium.webdriver.common.by import By

def click_by_xpath(xpath):
    driver.find_element(By.XPATH, xpath).click()
# driver.find_element(By.XPATH, '//div[text()="Learn"]').click()

click_by_xpath('//div[text()="Learn"]')
click_by_xpath('//div[text()="PLAY TUTORIAL"]')

In [66]:
#driver.find_element(By.XPATH, '//div[text()="PLAY TUTORIAL"]').click()
window_handles = driver.window_handles
new_window = None
for handle in window_handles:
    if handle != current_window:
        new_window = handle
        break

# Switch to the new window handle
if new_window:
    driver.switch_to.window(new_window)  # Step 4: Switch to the new tab
else: raise Exception('New tab not found')

while True:
    try:
        driver.find_elements(By.CLASS_NAME, "fa-minus-square-o")[1].click()
    except: break


In [178]:
handles = driver.window_handles
driver.switch_to.window(handles[2])

In [190]:
def get_player_tables():
    return driver.find_elements(By.CLASS_NAME, "player-table")

def get_num_players():
    return len(get_player_tables())

def parse_player_table(player_table):
    player_data = {}

    # Get child with name "player-name-wrapper"
    player_name_wrapper = player_table.find_element(By.CLASS_NAME, "player-name-wrapper")
    player_name = player_name_wrapper.find_element(By.CLASS_NAME, "player-name").text
    player_data["player_name"] = player_name

    # Get all children of class="line"
    player_lines = player_table.find_elements(By.CLASS_NAME, "line")
    player_data["lines"] = []
    for line in player_lines:
        line_data = {}
        index = int(line.get_attribute("id")[-1])
        if index >= 6: continue
        line_data["index"] = index if index != 0 else "floor"
        line_data["tiles"] = []
        children = line.find_elements(By.CLASS_NAME, "tile")
        for child in children:
            tile_index = int(child.get_attribute("class")[-1])
            line_data["tiles"].append(tile_index)
        player_data["lines"].append(line_data)

    # Get "wall" class div
    wall_data = []
    player_wall = player_table.find_element(By.CLASS_NAME, "wall")
    children = player_wall.find_elements(By.CLASS_NAME, "wall-spot")
    for child in children:
        tile_index = child.get_attribute("id")[-3:].split("-")
        x, y = int(tile_index[0]), int(tile_index[1])
        print(x, y)
        has_tiles = child.find_elements(By.CLASS_NAME, "tile")
        if has_tiles:
            wall_data.append({"x": x, "y": y})

    player_data["wall"] = wall_data

    return player_data

player_tables = get_player_tables()
for player_table in player_tables:
    data = parse_player_table(player_table)
    import json
    print(json.dumps(data, indent=4))
    break

1 1
1 2
1 3
1 4
1 5
2 1
2 2
2 3
2 4
2 5
3 1
3 2
3 3
3 4
3 5
4 1
4 2
4 3
4 4
4 5
5 1
5 2
5 3
5 4
5 5
{
    "player_name": "KANSKY",
    "lines": [
        {
            "index": 1,
            "tiles": []
        },
        {
            "index": 2,
            "tiles": []
        },
        {
            "index": 3,
            "tiles": []
        },
        {
            "index": 4,
            "tiles": []
        },
        {
            "index": 5,
            "tiles": [
                2,
                2,
                2
            ]
        },
        {
            "index": "floor",
            "tiles": []
        }
    ],
    "wall": [
        {
            "x": 1,
            "y": 2
        },
        {
            "x": 1,
            "y": 3
        },
        {
            "x": 1,
            "y": 4
        },
        {
            "x": 2,
            "y": 2
        },
        {
            "x": 2,
            "y": 3
        },
        {
            "x": 2,
            "y"

In [140]:
# def get_factories():
#     factories = []
#     i = 0
#     while True:
#         try:
#             factories.append(driver.find_element(By.XPATH, f'//div[@id="factory{i}"]'))
#         except: break
#         i += 1
    
#     factory_tiles = []
#     for factory in factories:
#         class_name = factory.get_attribute('class')
#         print(class_name)
#         # Get children divs
#         children = factory.find_elements(By.XPATH, './/*')
#         factory_tiles.append([])
#         for child in children:
#             tile_index = int(child.get_attribute('class').split(' ')[1][-1])
#             factory_tiles[-1].append(tile_index)

#     tiles = [[0] * 6 for _ in factory_tiles]
#     for i, factory in enumerate(factory_tiles):
#         for tile in factory:
#             tiles[i][tile] += 1

#     center_factory = tiles[0]
#     tile_taken_from_center = center_factory[0] != 1

#     center_factory = tuple(center_factory[1:])
#     other_factories = [tuple(tile[1:]) for tile in tiles[1:]]

#     return center_factory, other_factories, tile_taken_from_center

def get_factories():
    all_factory_elements = driver.find_elements(By.XPATH, '//div[@class="factory"]')
    all_factory_elements += driver.find_elements(By.XPATH, '//div[@class="factory-center"]')

    factories = []
    # Parse the factories
    for factory_element in all_factory_elements:
        factory = {}
        class_name = factory_element.get_attribute('class')
        factory["is_center"] = "center" in class_name
        children = factory_element.find_elements(By.XPATH, './/*')

        factory["tiles"] = []
        for child in children:
            tile_id = int(child.get_attribute('class').split(' ')[1][-1])
            factory["tiles"].append(tile_id)

        factories.append(factory)

    return factories

factories = get_factories()
import json
print(json.dumps(factories, indent=4))

[
    {
        "is_center": false,
        "tiles": []
    },
    {
        "is_center": false,
        "tiles": []
    },
    {
        "is_center": false,
        "tiles": []
    },
    {
        "is_center": false,
        "tiles": []
    },
    {
        "is_center": false,
        "tiles": []
    },
    {
        "is_center": false,
        "tiles": [
            5,
            4,
            3,
            5
        ]
    },
    {
        "is_center": false,
        "tiles": []
    },
    {
        "is_center": true,
        "tiles": [
            3,
            4,
            5,
            4,
            3,
            5,
            3,
            5,
            4
        ]
    }
]


In [62]:
factories = driver.find_elements(By.CLASS_NAME, "factory")
factories

[<selenium.webdriver.remote.webelement.WebElement (session="b71da789-2168-4e7e-855b-c045e08dfdaf", element="39c4bd05-06a7-4e14-8811-d86a758a691e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b71da789-2168-4e7e-855b-c045e08dfdaf", element="6fe01b8e-e9c4-414f-8333-9d6fbbed223c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b71da789-2168-4e7e-855b-c045e08dfdaf", element="fcc5dd6d-7a53-430b-b59e-6346f49c83ed")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b71da789-2168-4e7e-855b-c045e08dfdaf", element="8e69296e-5972-4c0b-ad2c-003d66442ddf")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b71da789-2168-4e7e-855b-c045e08dfdaf", element="36dab233-0a88-4a4b-8089-d47b72b6e31f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b71da789-2168-4e7e-855b-c045e08dfdaf", element="2786b030-6ef9-4b14-9261-aa65eefbbb26")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b71da789-2168-4e7e-855b-c045e08dfdaf", element

In [35]:
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [41]:
# from bs4 import BeautifulSoup
# soup = BeautifulSoup(driver.page_source, 'html.parser')

# # Store soup in a file
# with open("soup.html", "w") as file:
#     file.write(str(soup))

In [46]:
soup.find_all("div", class_="tile")

[]